# Práctica optativa

## Descripción

Vamos a intentar solucionar el problema de la mochila por fuerza bruta y utilizando un algoritmo genético.


Nuestra mochila va a tener una capacidad de **6404180** kilos. La **lista de objetos, pesos y valores** será:

['Hacha', 32252, 68674],

['Moneda de bronce', 225790, 471010],

['Corona', 468164, 944620],

['Estatua de diamante', 489494, 962094],

['Cinturón de esmeralda belt', 35384, 78344],

['Fósil', 265590, 579152],

['Moneda de oro', 497911, 902698],

['Casco', 800493, 1686515],

['Tinta', 823576, 1688691],

['Cofre de joyas', 552202, 1056157],

['Cuchillo', 323618, 677562],

['Espada', 382846, 833132],

['Máscara', 44676, 99192],

['Collar', 169738, 376418],

['Insignia', 610876, 1253986],

['Perlas', 854190, 1853562],

['Carcaj', 671123, 1320297],

['Anillo de rubí', 698180, 1301637],

['Pulsera de plata', 446517, 859835],

['Reloj', 909620, 1677534],

['Uniforme', 904818, 1910501],

['Veneno', 730061, 1528646],

['Bufanda de lana', 931932, 1827477],

['Arco', 952360, 2068204],

['Libro', 926023, 1746556],

['Copa de zinc', 978724, 2100851, 0]



**La idea es encontrar la combinación que nos permita llevar objetos de más valor.**

## Fuerza bruta

In [11]:
import time
import itertools # https://docs.python.org/3/library/itertools.html
from itertools import product 

# Set the indexes for data array
# 0 = name, 1 = weight, 2 = value, 3 = fitness
KNAPSACK_WEIGHT_INDEX = 1
KNAPSACK_VALUE_INDEX = 2
KNAPSACK_FITNESS_INDEX = 3

OBJETOS_POSIBLES = [['Hacha', 32252, 68674],

['Moneda de bronce', 225790, 471010],

['Corona', 468164, 944620],

['Estatua de diamante', 489494, 962094],

['Cinturón de esmeralda belt', 35384, 78344],

['Fósil', 265590, 579152],

['Moneda de oro', 497911, 902698],

['Casco', 800493, 1686515],

['Tinta', 823576, 1688691],

['Cofre de joyas', 552202, 1056157],

['Cuchillo', 323618, 677562],

['Espada', 382846, 833132],

['Máscara', 44676, 99192],

['Collar', 169738, 376418],

['Insignia', 610876, 1253986],

['Perlas', 854190, 1853562],

['Carcaj', 671123, 1320297],

['Anillo de rubí', 698180, 1301637],

['Pulsera de plata', 446517, 859835],

['Reloj', 909620, 1677534],

['Uniforme', 904818, 1910501],

['Veneno', 730061, 1528646],

['Bufanda de lana', 931932, 1827477],

['Arco', 952360, 2068204],

['Libro', 926023, 1746556],

['Copa de zinc', 978724, 2100851]]

1) Calcula la aptitud de cada solución basándote en el valor total y en si viola la restricción del peso máximo de la mochila.

In [12]:
def calcular_aptitud(solution, maximum_weight):
    total_weight = 0
    total_value = 0
    # Get the values and weight for each item marked with a 1
    for item_index in range(0, len(solution)):
        item = solution[item_index]
        if item == 1:
            total_weight += OBJETOS_POSIBLES[item_index][KNAPSACK_WEIGHT_INDEX]
            total_value += OBJETOS_POSIBLES[item_index][KNAPSACK_VALUE_INDEX]
    # Zero fitness if the weight constraint is violated
    if total_weight > maximum_weight:
        return 0
    return total_value
    
# Get all possible combinations of items. This is exhaustive and computationally expensive!
def get_all_combinations(items):
    combinations = []
    for index in range(0, len(items)):
        combinations.append(items[index])
        possibilities = [list(x) for x in itertools.combinations(items, index)]
        combinations.append(possibilities)
    return combinations

2) Implementa un algoritmo de fuerza bruta, esto es:

- Obtén una solución.
- Calcula su aptitud.
- Imprime la mejor solución al final del proceso.

En lugar de calcular a priori todas las soluciones, lo que resulta computacionalmente complejo, vamos a utilizar la siguiente expresión:

    for i in product([0, 1], repeat=bit_size):

Que nos iterará sobre todas las posibles soluciones de acuerdo con sus valores binarios.
    

In [13]:
def ejecutar_fuerza_bruta():
    bit_string_size = 8
    best_score = 0
    best_individual = []
    knapsack_max_capacity = 10
    print('Number of combinations: ', 2**bit_string_size)
    iteration = 0
    for i in product([0, 1], repeat=bit_string_size):
        current = calcular_aptitud(i, knapsack_max_capacity)
        if current > best_score:
            best_score = current
            best_individual = i
            print('Iteration: ', iteration)
            print('Best score: ', best_score)
            print('Best individual: ', best_individual)
        iteration += 1
    

3) Ejecuta la siguiente celda para obtener el resultado del algoritmo.

In [14]:
start_time = time.time()
ejecutar_fuerza_bruta()
end_time = time.time()
total_time = end_time - start_time
print('Tiempo total: ', total_time)

Number of combinations:  256
Tiempo total:  0.0029938220977783203


## Algoritmos genéticos

Es necesario configurar el algoritmo y seguir el ciclo de vida de los algoritmos genéticos.

Parámetros:

- Condición de parada = 1200 generaciones
- Tamaño de la población = 1000
- Selección de padres = Ruleta
- Reproducción basada recombinación en dos puntos: 10 y 20
- Ratio de mutación: 15
- Selección de la población: 80% hijos y 20% más aptos de la anterior generación

Ciclo de vida

- Crear una población
- Seleccionar padres basados en aptitud.
- Reproducir individuos usando los padres seleccionados: Reproducir y mutar
- Poblar la siguiente generación



In [15]:
import random

PESO_MAXIMO = 6404180
POSICION_NOMBRE_OBJETO = 0
POSICION_PESO_OBJETO = 1
POSICION_VALOR_OBJETO = 2

POBALCION = 100
PORCENTAJE_POBLACION_HIJOS = 0.9
PORCENTAJE_APTOS_NECESARIA_INICIAL = 0.5
PORCENTAJE_MAS_APTOS = 0.2

PESO_MEDIO = None
VALOR_MEDIO = None

CONVERSION_APTITUD = 0

def calcular_media_peso():
    global PESO_MEDIO
    peso_total = 0
    for objeto in OBJETOS_POSIBLES:
         peso_total = peso_total + objeto[POSICION_PESO_OBJETO]
    PESO_MEDIO = peso_total / 25

def calcular_media_valor():
    global VALOR_MEDIO
    valor_total = 0
    for objeto in OBJETOS_POSIBLES:
         valor_total = valor_total + objeto[POSICION_VALOR_OBJETO]
    VALOR_MEDIO = valor_total / 25

class Objeto:
    
    def __init__(self, nombre, peso, valor, objeto_padre_1, objeto_padre_2):
        self.nombre = nombre
        self.peso = peso
        self.valor = valor
        self.aptitud = 0
        if (objeto_padre_1 != None):
            self.nombre_padre_1 = objeto_padre_1.nombre
            self.nombre_padre_2 = objeto_padre_2.nombre
        else:
            self.nombre_padre_1 = None
            self.nombre_padre_2 = None
        # self.determinar_aptitud()
        self.determinar_aptitud_avanzada()

    def determinar_aptitud(self):
        self.aptitud = self.valor / self.peso # cuanto mayor sea la aptitud, mas vale respecto a peso
        print(f"Objeto: {self.nombre}:{self.peso}, {self.valor}  ------>  Aptitud: {self.aptitud} --- Objetos padres:{self.nombre_padre_1, self.nombre_padre_1}")
    
    def determinar_aptitud_avanzada(self):
        global PESO_MEDIO
        global VALOR_MEDIO
        global CONVERSION_APTITUD
        if (self.peso < PESO_MEDIO):
            self.aptitud = self.aptitud + ((PESO_MEDIO - self.peso) * CONVERSION_APTITUD)
        if (self.valor > VALOR_MEDIO):
            self.aptitud = self.aptitud + (self.valor - VALOR_MEDIO) 
        print(f"Objeto: {self.nombre}:{self.peso}, {self.valor}  ------>  Aptitud: {self.aptitud} --- Objetos padres:{self.nombre_padre_1, self.nombre_padre_1}")


In [ ]:
### Paso 1: Crear una población

In [16]:
def generar_lista_poblacion():
    print("Generando lista")
    poblacion = []
    for i in range(POBALCION):
        objeto = random.choice(OBJETOS_POSIBLES)
        nuevo_objeto = Objeto(objeto[POSICION_NOMBRE_OBJETO], objeto[POSICION_PESO_OBJETO], objeto[POSICION_VALOR_OBJETO], None, None)
        poblacion.append(nuevo_objeto)
    return poblacion

### Paso 2: Seleccionar padres

In [17]:
# Devolvemos los 50 mejores
def devolver_cromosomas_aptos(lista_poblacion):
    print("Devolviendo cromosomas aptos")
    lista_poblacion.sort(key=lambda x: x.aptitud, reverse=True)
    numero_lista_aptos_necesaria = devolver_numero_necesario_segun_porcentaje(lista_poblacion, PORCENTAJE_APTOS_NECESARIA_INICIAL) # 50
    lista_aptos = lista_poblacion[:numero_lista_aptos_necesaria]
    return lista_aptos

def devolver_numero_necesario_segun_porcentaje(lista, porcentaje):
    numero_poblacion = len(lista)
    nuemro_necesario = int(numero_poblacion * porcentaje)
    return nuemro_necesario

### Paso 3: Reproducir individuos

In [18]:
def reproducir_hijos(lista_objetos_aptos, rango_lista_hijos):
    lista_hijos = []
    for i in range(rango_lista_hijos):
        objeto_padre_1 = lista_objetos_aptos[random.randrange(len(lista_objetos_aptos))]
        objeto_padre_2 = lista_objetos_aptos[random.randrange(len(lista_objetos_aptos))]
        lista_hijos.append(mutacion(objeto_padre_1, objeto_padre_2))
    return lista_hijos

def mutacion(objeto_padre_1, objeto_padre_2):
    objeto_hijo = None
    if (objeto_padre_1.aptitud > objeto_padre_2.aptitud):
        objeto_hijo = Objeto(
            nombre = objeto_padre_1.nombre, 
            peso = objeto_padre_1.peso, 
            valor = objeto_padre_1.valor, 
            objeto_padre_1 = objeto_padre_1, 
            objeto_padre_2 = objeto_padre_2)
    else:
        objeto_hijo = Objeto(
            nombre = objeto_padre_2.nombre, 
            peso = objeto_padre_2.peso, 
            valor = objeto_padre_2.valor, 
            objeto_padre_1 = objeto_padre_1, 
            objeto_padre_2 = objeto_padre_2)
    return objeto_hijo

### Paso 4: Poblar la siguiente generación

In [19]:
def generar_proxima_generacion(lista_objetos_aptos, lista_poblacion):
    print("Generando proxima generacion")
    rango_mas_aptos = devolver_numero_necesario_segun_porcentaje(lista_objetos_aptos, PORCENTAJE_MAS_APTOS) # 10
    rango_lista_hijos = devolver_numero_necesario_segun_porcentaje(lista_poblacion, PORCENTAJE_POBLACION_HIJOS) # 90
    lista_proxima_generacion = lista_objetos_aptos[:rango_mas_aptos] # ya tenemos el 10%
    lista_hijos = reproducir_hijos(lista_objetos_aptos, rango_lista_hijos) # Otro 90%
    lista_proxima_generacion.extend(lista_hijos)
    lista_proxima_generacion.sort(key=lambda x: x.aptitud, reverse=True)
    return lista_proxima_generacion

### Crea un script y ejecuta los pasos anteriores teniendo en cuenta los parámetros del algoritmo.

In [20]:
def calcular_peso_total_nueva_generacion(lista_poblacion):
    peso_total = 0
    for objeto in lista_poblacion:
         peso_total = peso_total + objeto.peso
    return peso_total

def calcular_conversion_aptitud():
    global PESO_MEDIO
    global VALOR_MEDIO
    global CONVERSION_APTITUD
    CONVERSION_APTITUD = VALOR_MEDIO / PESO_MEDIO

def algoritmo_genetico():
    global PESO_MAXIMO
    global PESO_MEDIO
    global VALOR_MEDIO
    peso_total = 100000000000
    pasos = 0
    calcular_media_peso()
    calcular_media_valor()
    calcular_conversion_aptitud()
    lista_poblacion = generar_lista_poblacion()
    lista_objetos_aptos = None
    lista_proxima_generacion = None
    while peso_total > PESO_MAXIMO or pasos < 20:
        # PESO_MEDIO = PESO_MEDIO - 10000
        # VALOR_MEDIO = VALOR_MEDIO + 50000
        lista_objetos_aptos = devolver_cromosomas_aptos(lista_poblacion)  # 50% aptos
        lista_proxima_generacion = generar_proxima_generacion(lista_objetos_aptos, lista_poblacion)
        lista_poblacion.clear()
        lista_poblacion = lista_proxima_generacion.copy()
        peso_total = calcular_peso_total_nueva_generacion(lista_poblacion)
        lista_objetos_aptos.clear()
        lista_proxima_generacion.clear()
        pasos = pasos + 1

algoritmo_genetico()

Generando lista
Objeto: Libro:926023, 1746556  ------>  Aptitud: 551622.2 --- Objetos padres:(None, None)
Objeto: Moneda de bronce:225790, 471010  ------>  Aptitud: 736898.3390501718 --- Objetos padres:(None, None)
Objeto: Bufanda de lana:931932, 1827477  ------>  Aptitud: 632543.2 --- Objetos padres:(None, None)
Objeto: Cuchillo:323618, 677562  ------>  Aptitud: 538445.3824365052 --- Objetos padres:(None, None)
Objeto: Anillo de rubí:698180, 1301637  ------>  Aptitud: 106703.19999999995 --- Objetos padres:(None, None)
Objeto: Pulsera de plata:446517, 859835  ------>  Aptitud: 289133.6320699125 --- Objetos padres:(None, None)
Objeto: Moneda de bronce:225790, 471010  ------>  Aptitud: 736898.3390501718 --- Objetos padres:(None, None)
Objeto: Reloj:909620, 1677534  ------>  Aptitud: 482600.19999999995 --- Objetos padres:(None, None)
Objeto: Estatua de diamante:489494, 962094  ------>  Aptitud: 201950.89587592357 --- Objetos padres:(None, None)
Objeto: Arco:952360, 2068204  ------>  Aptit

Objeto: Hacha:32252, 68674  ------>  Aptitud: 1129507.697486364 --- Objetos padres:('Hacha', 'Hacha')
Objeto: Hacha:32252, 68674  ------>  Aptitud: 1129507.697486364 --- Objetos padres:('Hacha', 'Hacha')
Objeto: Hacha:32252, 68674  ------>  Aptitud: 1129507.697486364 --- Objetos padres:('Hacha', 'Hacha')
Objeto: Hacha:32252, 68674  ------>  Aptitud: 1129507.697486364 --- Objetos padres:('Hacha', 'Hacha')
Objeto: Hacha:32252, 68674  ------>  Aptitud: 1129507.697486364 --- Objetos padres:('Hacha', 'Hacha')
Objeto: Hacha:32252, 68674  ------>  Aptitud: 1129507.697486364 --- Objetos padres:('Hacha', 'Hacha')
Objeto: Hacha:32252, 68674  ------>  Aptitud: 1129507.697486364 --- Objetos padres:('Hacha', 'Hacha')
Objeto: Hacha:32252, 68674  ------>  Aptitud: 1129507.697486364 --- Objetos padres:('Hacha', 'Hacha')
Objeto: Hacha:32252, 68674  ------>  Aptitud: 1129507.697486364 --- Objetos padres:('Hacha', 'Hacha')
Objeto: Hacha:32252, 68674  ------>  Aptitud: 1129507.697486364 --- Objetos padres